In [2]:
import numpy as np
import scipy.sparse as sparse
import random
import hashlib

# Load in Data

In [3]:
data = np.load('Data/user_movie_rating.npy')[:,0:2]

In [4]:
col = data[:, 0]
row = data[:, 1]

n_col = col.max() + 1
n_row = row.max() + 1 

M = sparse.coo_matrix((np.ones(len(data)), (row, col)), shape=(n_row, n_col)).tocsr()
small_M  = M[:,0:1000]

In [122]:
perm = 150 
perm_seeds = []
row_len,_ = small_M.get_shape()
seed = 100000
for i in range(perm):
    perm_seeds.append(int(hashlib.sha256(str(seed+i).encode()).hexdigest(),16)%4294967295)


In [ ]:
from tqdm import tqdm

##TODO: Implement multithreading so we can calculate the seeds in parallel


def minhashing(M):
    signature = []
    index_perm = np.arange(M.shape[0])#reshuffeling the same index array
    sig_len = M.shape[1]-1
    for seed in tqdm(perm_seeds):
        random.seed(seed)
        np.random.shuffle(index_perm)
        perm_M = M[index_perm[:1000],1:] #Just take the top 1000 because it is very likely for each user to be in the first thousand movies at least once
        non_zero = perm_M.nonzero()
        sig = non_zero[0][np.unique(non_zero[1], return_index=True)[1]]
        if len(sig)!=sig_len:
            ##TODO Handle if for one user no movie can be found in the top k
            break
        signature.append(sig)        
    return signature
        

        


In [124]:
a = minhashing(M)

100%|██████████| 150/150 [00:57<00:00,  2.62it/s]


In [ ]:
def LSH():
    pass
# Devide Signatures into b bands and r rows
# implement hash table to check for identity in linear time

        


In [120]:
def approx_prob_all_users(U = M.shape[1], N=M.shape[0], k=300, n=1000):
    p_none = (1 - k/N) ** n
    return (1 - p_none) ** U

In [121]:
approx_prob_all_users()

0.9958237747065899